# Heisenberg Hamiltonian Tapering Example
First, build the Heisenberg Hamiltonian
\begin{equation}
H = \sum_{i=1}^{N-1} (J_X X^{(i)} X^{(i+1)} + J_Y Y^{(i)} Y^{(i+1)} + J_Z Z^{(i)} Z^{(i+1)} + h Z^{(i)})
\end{equation}
where $J_X, J_Y, J_Z \in \mathbb{R}$ are coupling constants and $h \in \mathbb{R}$ the strength of an external magnetic field.

In [74]:
import numpy as np
from symmer.symplectic import PauliwordOp, QuantumState
from symmer.utils import exact_gs_energy

def place_sites(indices, pauli, N):
    I = ['I']*N
    for i in indices:
        I[i] = pauli
    return ''.join(I)
    
def HeisenbergHam(N, J_X=1, J_Y=1, J_Z=1, h=1):
    constants = {'X':J_X, 'Y':J_Y, 'Z':J_Z, 'h':h}
    H_dict = {}
    for i in range(N-1):
        for P in ['X','Y','Z']:
            H_dict[place_sites([i, i+1], P, N)] = constants[P]
    for i in range(N):
        H_dict[place_sites([i], 'Z', N)] = constants['h']

    return PauliwordOp.from_dictionary(H_dict).cleanup().multiply_by_constant(-1/2).sort()

J_X, J_Y, J_Z, h = np.random.random(4)
H = HeisenbergHam(4, J_X, J_Y, J_Z, h=0)
gs_nrg, gs_vec = exact_gs_energy(H.to_sparse_matrix)
gs_psi = QuantumState.from_array(gs_vec).cleanup(zero_threshold=1e-5)
print('The heisenberg Hamiltonian is\n')
print(H); print()
print(f'with ground state energy {gs_nrg} and corresponding eigenvector\n')
print(gs_psi)

The heisenberg Hamiltonian is

-0.482+0.000j IIXX +
-0.482+0.000j IXXI +
-0.482+0.000j XXII +
-0.471+0.000j IIYY +
-0.471+0.000j IYYI +
-0.471+0.000j YYII +
-0.032+0.000j IIZZ +
-0.032+0.000j IZZI +
-0.032+0.000j ZZII

with ground state energy -2.0743038562181226 and corresponding eigenvector

-0.005+0.000j |0000> +
-0.231+0.000j |0011> +
-0.495+0.000j |0101> +
-0.448+0.000j |0110> +
-0.448+0.000j |1001> +
-0.495+0.000j |1010> +
-0.231+0.000j |1100> +
-0.005+0.000j |1111>


We may perform a VQE simulation over this space:

# Taper the Hamiltonian

In [75]:
from symmer.projection import QubitTapering

QT = QubitTapering(H)

print(f'Qubit tapering permits a reduction of {H.n_qubits} -> {H.n_qubits-QT.n_taper} qubits.\n')
print('The following symmetry generators were identified:\n')
print(QT.symmetry_generators); print()
print('which we may rotate onto the single-qubit Pauli operators\n') 
print(QT.symmetry_generators.rotate_onto_single_qubit_paulis()); print()
print('via a sequence of Clifford operations R_k = e^{i pi/4 P_k} where:\n')
for index, (P_k, angle) in enumerate(QT.symmetry_generators.stabilizer_rotations):
    P_k.sigfig=0
    print(f'P_{index} = {P_k}')

Qubit tapering permits a reduction of 4 -> 2 qubits.

The following symmetry generators were identified:

 1 ZZZZ 
 1 XXXX

which we may rotate onto the single-qubit Pauli operators

-1 IXII 
-1 XIII

via a sequence of Clifford operations R_k = e^{i pi/4 P_k} where:

P_0 =  1+0j YZZZ
P_1 =  1+0j XYXX
P_2 =  1+0j IYII


In [76]:
H_taper = QT.taper_it()

print('We rotate the Hamiltonian accordingly:\n')
print(H.perform_rotations(QT.symmetry_generators.stabilizer_rotations)); print()
print(f'and observe that qubit positions {QT.stab_qubit_indices} consist only of Pauli I, {QT.target_sqp} operators.\n')
print(f'These may therefore be removed to yield a {H.n_qubits-QT.n_taper}-qubit reduced Hamiltonian:\n')
print(H_taper)

We rotate the Hamiltonian accordingly:

-0.032+0.000j IIZZ +
-0.482+0.000j IIXX +
-0.471+0.000j IIYY +
 0.032-0.000j IXZI +
 0.482-0.000j IXXX +
 0.032-0.000j XIZZ +
 0.471-0.000j XIZX +
 0.482-0.000j XXIX +
-0.471+0.000j XXYY

and observe that qubit positions [1 0] consist only of Pauli I, X operators.

These may therefore be removed to yield a 2-qubit reduced Hamiltonian:

-0.032+0.000j ZI +
-0.065+0.000j ZZ +
 0.482+0.000j IX +
-0.471+0.000j ZX +
-0.965+0.000j XX +
-0.942+0.000j YY


The power of qubit tapering is that it _exactly_ preserves the ground state energy, as we can see here:

In [78]:
gs_nrg_tap, gs_nrg_vec = exact_gs_energy(H_taper.to_sparse_matrix)
gs_psi_tap = QuantumState.from_array(gs_nrg_vec)

print(f'The ground state energy of the Hamiltonian is {gs_nrg}')
print(f'and for the 2-qubit tapered Hamiltonian it is {gs_nrg_tap};')
print(f'the energy error is {abs(gs_nrg - gs_nrg_tap)}.\n')
print('The tapered ground state is:\n')
print(gs_psi_tap)

The ground state energy of the Hamiltonian is -2.0743038562181226
and for the 2-qubit tapered Hamiltonian it is -2.0743038562181217;
the energy error is 8.881784197001252e-16.

The tapered ground state is:

 0.007+0.000j |00> +
-0.634+0.000j |01> +
-0.701+0.000j |10> +
 0.327+0.000j |11>
